# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [102]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

pd.set_option('display.max_rows', 1000)

nhl_df=pd.read_csv("assests/nhl.csv")
cities=pd.read_html("assests/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_df(my_df):
    # define a new dataFrame for win/loss ratio
    my_df['team'].replace({'\\*': ''}, regex=True, inplace = True) #delete *
    # transfer string to int
    # https://datatofish.com/string-to-integer-dataframe/
    my_df['W'] = pd.to_numeric(my_df['W'], errors='coerce')
    my_df['L'] = pd.to_numeric(my_df['L'], errors='coerce')

    #group-by-and-sum to sum win and loss
    #https://stackoverflow.com/questions/39922986/pandas-group-by-and-sum
    team_win = my_df.groupby(['team'])['W'].agg('sum')
    team_loss = my_df.groupby(['team'])['L'].agg('sum')

    #concat into a new datFrame
    #https://www.datacamp.com/community/tutorials/joining-dataframes-pandas
    team_df = pd.concat([team_win,team_loss],axis=1)

    team_df['WL_ratio'] = team_df['W']/team_df['L']#add win/loss ratio

    #https://stackoverflow.com/questions/15205536/how-to-escape-asterisk-in-regexp
    team_df['name'] = team_df.index #put index into a column
    team_df['city'] = team_df['name'].replace({r"\([^()]*\)": ''}, regex=True) #remove text within parentheses
    team_df['city'] = team_df['city'].str.strip() #remove white spaces at both ends
    team_df['city'] = team_df['city'].replace({'\s(\w+)$': ''}, regex=True) #delete the last word

    team_df.replace({"city": # replace city name
                               {'Anaheim':'Los Angeles',
                                'Arizona':'Phoenix',
                                'Boston Red':'Boston',
                                'Carolina':'Raleigh',
                                'Chicago White':'Chicago',
                                'Colorado':'Denver',
                                "Columbus Blue": "Columbus",
                                'Dallas':'Dallas–Fort Worth',
                                'Detroit Red':'Detroit',
                                'Florida':'Miami–Fort Lauderdale',
                                'Golden State':'San Francisco Bay Area',
                                'Indiana':'Indianapolis',
                                'Los Angeles Angels of':'Los Angeles',
                                'Portland Trail':'Portland',
                                'Miami':'Miami–Fort Lauderdale',
                                'Minnesota':'Minneapolis–Saint Paul',
                                'New Jersey':'New York City',
                                "New York":"New York City",
                                'San Francisco':'San Francisco Bay Area',
                                'San Jose':'San Francisco Bay Area',
                                'Tampa Bay':'Tampa Bay Area',
                                'Texas':'Dallas–Fort Worth',
                                'Toronto Blue':'Toronto',
                                "Toronto Maple": "Toronto",
                                'Utah':'Salt Lake City',
                                "Vegas Golden": "Las Vegas",
                                'Washington':'Washington, D.C.'}}, inplace = True)
    raw_df = pd.merge(cities,team_df,how = 'inner', left_on = 'Metropolitan area', right_on ='city' )
    
    result_df = raw_df.groupby(by=['Metropolitan area', 'Population (2016 est.)[8]'], as_index=False).mean()
    result_df['Population (2016 est.)[8]'] = result_df['Population (2016 est.)[8]'].astype(int)
    
    return result_df

type(clean_df(nhl_df)['Population (2016 est.)[8]'][1])

numpy.int32

In [103]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assests/nhl.csv")
cities=pd.read_html("assests/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE

    
    merged_df = clean_df(nhl_df)
    population_by_region = merged_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['WL_ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    print(len(population_by_region))
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()
nhl_correlation()

28


(0.1723994489168242, 0.38034795242610253)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [104]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assests/nba.csv")
cities=pd.read_html("assests/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    merged_df = clean_df(nba_df)
    
    population_by_region = merged_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    
    
    win_loss_by_region = merged_df['WL_ratio'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
nba_correlation()

(-0.0432452998268566, 0.8270383291153282)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [105]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assests/mlb.csv")
cities=pd.read_html("assests/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    merged_df = clean_df(mlb_df)
    population_by_region = merged_df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['WL_ratio'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    print(len(population_by_region))
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

mlb_correlation()

26


(0.24047284316076228, 0.23667692968141837)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [107]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assests/nfl.csv")
cities=pd.read_html("assests/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    merged_df = clean_df(nfl_df)
    population_by_region = merged_df['Population (2016 est.)[8]']  # pass in metropolitan area population from cities
    win_loss_by_region = merged_df['WL_ratio'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)
nfl_correlation()

(-0.18723502552726992, 0.33076370670343513)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values